# Pre-trained Word Embeddings in PyTorch

In [6]:
import numpy as np

import torch
import torch.nn as nn

## Load Pre-trained Word Embeddings (GloVe)

In [9]:
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']

# define class labels
labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [27]:
# join all the sentences together and extract the unique characters from the combined sentences
words = set(' '.join(docs).split())

# creating a dictionary that maps integers to the words
int2word = dict(enumerate(words))

# creating another dictionary that maps characters to integers
word2int = { word: idx for idx, word in int2word.items() }

# calculate vocab size
vocab_size = len(words)

In [ ]:
# load the whole pre-trained word embedding into memory
embeddings_glove = dict()
f = open('./embeddings/glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0] # you dan filter the embedding for the unique words in your training data to make it faster
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_glove[word] = coefs

f.close()
print('Loaded %s word vectors from GloVe.' % len(embeddings_glove))

In [35]:
# create the weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in word2int.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Implement Embeddings

In [3]:
def create_embedding_layer(embedding_matrix, non_trainable=False):
    
    n_embeddings, embedding_dim = embedding_matrix.size()
    embedding_layer = nn.Embedding(n_embeddings, embedding_dim)
    embedding_layer.load_state_dict({'weight': embedding_matrix})
    
    if non_trainable:
        embedding_layer.weight.requires_grad = False
        
    return embedding_layer, n_embeddings, embedding_dim

In [4]:
class GRU(nn.Module):
    
    # embedding_matrix includes input_size and embedding_dim
    def __init__(self, embedding_matrix, hidden_dim, output_size, n_layers):
        super(model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.embedding_layer, self.n_embeddings, self.embedding_dim = create_embedding_layer(embedding_matrix, True)        
        self.gru_layer = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True)
        self.relu = nn.ReLU()
        self.fc_layer = nn.Linear(hidden_dim, output_size)
        
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return Variable(hidden)
    
    def forward(self, x, hidden):
        
        batch_size = x.size(0)
        
        x = x.long()
        embeds = self.embedding_layer(x)
        gru_out, hidden = self.gru_layer(embeds, hidden)
        output = self.fc_layer(self.relu(gru_out[:,-1]))
        
        return output, hidden

In [ ]:
gru = GRU(embedding_matrix, hidden_dim=256, output_size=1, n_layers=2)

---